<a href="https://colab.research.google.com/github/thegayankalinga/Software-Effort-Estimation-Model/blob/main/see_implementation_v5_p3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Phase 3: Deep Learning Implementation

### Setup & Load the Data

In [ ]:
# Install required libraries if not installed
!pip install pandas numpy tensorflow keras matplotlib seaborn scikit-learn --quiet

# Import necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, GRU
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score


# Define paths
DATA_PATH = "/content/drive/MyDrive/Projects/msc_project/results_data/"
MODELS_PATH = "/content/drive/MyDrive/Projects/msc_project/models/"
PERFORMANCE_DATA = "/content/drive/MyDrive/Projects/msc_project/performance_data/"

# Ensure directories exist
# os.makedirs(MODELS_PATH, exist_ok=True)
# os.makedirs(PERFORMANCE_DATA, exist_ok=True)

# Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

# Ensure Google Drive is mounted
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print(tf.__version__)

2.17.1


#### Load the Preprocessed Data

In [ ]:
# Load preprocessed data
X_train = pd.read_csv(os.path.join(DATA_PATH, "X_train.csv"))
X_val = pd.read_csv(os.path.join(DATA_PATH, "X_val.csv"))
X_test = pd.read_csv(os.path.join(DATA_PATH, "X_test.csv"))
y_train = pd.read_csv(os.path.join(DATA_PATH, "y_train.csv"))
y_val = pd.read_csv(os.path.join(DATA_PATH, "y_val.csv"))
y_test = pd.read_csv(os.path.join(DATA_PATH, "y_test.csv"))

# Convert data to NumPy arrays for TensorFlow
X_train, X_val, X_test = X_train.to_numpy(), X_val.to_numpy(), X_test.to_numpy()
y_train, y_val, y_test = y_train.to_numpy(), y_val.to_numpy(), y_test.to_numpy()

print(f"X_train Shape: {X_train.shape}, y_train Shape: {y_train.shape}")
print(f"X_val Shape: {X_val.shape}, y_val Shape: {y_val.shape}")
print(f"X_test Shape: {X_test.shape}, y_test Shape: {y_test.shape}")

X_train Shape: (35000, 41), y_train Shape: (35000, 4)
X_val Shape: (7500, 41), y_val Shape: (7500, 4)
X_test Shape: (7500, 41), y_test Shape: (7500, 4)


### Define Model Architecture

#### Define Multi-Layer Perceptron

In [ ]:
def build_mlp_model(input_shape, output_shape):
    model = Sequential([
        Dense(128, activation='relu', input_shape=(input_shape,)),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(output_shape, activation='linear')  # Linear activation for regression
    ])

    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# Initialize MLP model
mlp_model = build_mlp_model(X_train.shape[1], y_train.shape[1])

# Summary of the model
mlp_model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │           5,376 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 4)                   │             260 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,892 (54.27 KB)

 Trainable params: 13,892 (54.27 KB)

 Non-trainable params: 0 (0.00 B)

#### LSTM Model

In [ ]:
def build_lstm_model(input_shape, output_shape):
    model = Sequential([
        LSTM(128, activation='relu', return_sequences=True, input_shape=(1, input_shape)),
        LSTM(64, activation='relu'),
        Dense(output_shape, activation='linear')
    ])

    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# Reshape data for LSTM
X_train_lstm = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_val_lstm = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
X_test_lstm = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Initialize LSTM model
lstm_model = build_lstm_model(X_train.shape[1], y_train.shape[1])

# Summary of LSTM model
lstm_model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 1, 128)              │          87,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 4)                   │             260 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 136,708 (534.02 KB)

 Trainable params: 136,708 (534.02 KB)

 Non-trainable params: 0 (0.00 B)

### Train Models

In [ ]:
#use early stopping & learning rate scheduling to prevent overfitting

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001)

# Train MLP Model
history_mlp = mlp_model.fit(X_train, y_train, epochs=100, batch_size=32,
                            validation_data=(X_val, y_val), callbacks=[early_stopping, reduce_lr])

# Train LSTM Model
history_lstm = lstm_model.fit(X_train_lstm, y_train, epochs=100, batch_size=32,
                              validation_data=(X_val_lstm, y_val), callbacks=[early_stopping, reduce_lr])

Epoch 1/100
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - loss: 0.2581 - mae: 0.3690 - val_loss: 0.0452 - val_mae: 0.1678 - learning_rate: 0.0010
Epoch 2/100
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 0.0648 - mae: 0.2001 - val_loss: 0.0290 - val_mae: 0.1359 - learning_rate: 0.0010
Epoch 3/100
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.0459 - mae: 0.1682 - val_loss: 0.0187 - val_mae: 0.1074 - learning_rate: 0.0010
Epoch 4/100
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.0342 - mae: 0.1443 - val_loss: 0.0178 - val_mae: 0.1045 - learning_rate: 0.0010
Epoch 5/100
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0279 - mae: 0.1299 - val_loss: 0.0130 - val_mae: 0.0890 - learning_rate: 0.0010
Epoch 6/100
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0239 - mae: 0.1193 - val_loss: 0.0179 - val_mae: 0.1017 - learning_rate: 0.0010
Epoch 7/100
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.0195 - mae: 0.1075 - val_loss: 0.0245 - val_mae: 0.1198 - learnin

### Evaluate & Save Performance Data

In [ ]:
# Function to evaluate model
def evaluate_model(model, X, y_true, model_name):
    y_pred = model.predict(X)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    mmre = np.mean(np.abs((y_true - y_pred) / (y_true + 1e-10)))

    print(f"\n📌 {model_name} Model Evaluation:")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R-Squared: {r2:.4f}")
    print(f"MAPE: {mape:.2f}%")
    print(f"MMRE: {mmre:.4f}")

    return mse, rmse, r2, mape, mmre

# Evaluate models
mlp_metrics = evaluate_model(mlp_model, X_test, y_test, "MLP Model")
lstm_metrics = evaluate_model(lstm_model, X_test_lstm, y_test, "LSTM Model")

# Save Performance Data
performance_df = pd.DataFrame({
    "Model": ["MLP Model", "LSTM Model"],
    "MSE": [mlp_metrics[0], lstm_metrics[0]],
    "RMSE": [mlp_metrics[1], lstm_metrics[1]],
    "R-Squared": [mlp_metrics[2], lstm_metrics[2]],
    "MAPE (%)": [mlp_metrics[3], lstm_metrics[3]],
    "MMRE": [mlp_metrics[4], lstm_metrics[4]]
})

performance_df.to_csv(os.path.join(PERFORMANCE_DATA, "deep_learning_performance.csv"), index=False)

# # Save trained models
# mlp_model.save(os.path.join(MODELS_PATH, "mlp_model.h5"))
# lstm_model.save(os.path.join(MODELS_PATH, "lstm_model.h5"))

# Save models with their compiled state
# Save models in the recommended Keras format (.keras)

#Perform a dummy forward pass to build the model before saving
dummy_input = np.zeros((1, 1, X_train.shape[1]))  # Ensure correct input shape
_ = lstm_model.predict(dummy_input)  # This forces Keras to define layer inputs

# Save models in the recommended format
lstm_model.save(os.path.join(MODELS_PATH, "lstm_model.keras"), include_optimizer=True)


mlp_model.save(os.path.join(MODELS_PATH, "mlp_model.keras"), include_optimizer=True)
#lstm_model.save(os.path.join(MODELS_PATH, "lstm_model.keras"), include_optimizer=True)

print("\n✅ Deep learning models and performance data saved successfully.")

235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

📌 MLP Model Model Evaluation:
MSE: 0.0126
RMSE: 0.1124
R-Squared: 0.9873
MAPE: 209.26%
MMRE: 2.0926
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

📌 LSTM Model Model Evaluation:
MSE: 0.0000
RMSE: 0.0029
R-Squared: 1.0000
MAPE: 7.60%
MMRE: 0.0760
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 338ms/step

✅ Deep learning models and performance data saved successfully.


In [ ]:
# Function to evaluate model
def evaluate_model(model, X, y_true, model_name):
    y_pred = model.predict(X)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    mmre = np.mean(np.abs((y_true - y_pred) / (y_true + 1e-10)))

    print(f"\n📌 {model_name} Model Evaluation:")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R-Squared: {r2:.4f}")
    print(f"MAPE: {mape:.2f}%")
    print(f"MMRE: {mmre:.4f}")
    return mse, rmse, r2, mape, mmre

# Evaluate models
mlp_metrics = evaluate_model(mlp_model, X_test, y_test, "MLP Model")
lstm_metrics = evaluate_model(lstm_model, X_test_lstm, y_test, "LSTM Model")

# Save Performance Data
performance_df = pd.DataFrame({
    "Model": ["MLP Model", "LSTM Model"],
    "MSE": [mlp_metrics[0], lstm_metrics[0]],
    "RMSE": [mlp_metrics[1], lstm_metrics[1]],
    "R-Squared": [mlp_metrics[2], lstm_metrics[2]],
    "MAPE (%)": [mlp_metrics[3], lstm_metrics[3]],
    "MMRE": [mlp_metrics[4], lstm_metrics[4]]
})
performance_df.to_csv(os.path.join(PERFORMANCE_DATA, "deep_learning_performance.csv"), index=False)

# Perform dummy forward passes to build the models before saving
# For LSTM
dummy_input_lstm = np.zeros((1, 1, X_train_lstm.shape[2]))  # Shape: (batch_size, timesteps, features)
_ = lstm_model.predict(dummy_input_lstm)

# For MLP
dummy_input_mlp = np.zeros((1, X_train.shape[1]))  # Shape: (batch_size, features)
_ = mlp_model.predict(dummy_input_mlp)

# Save models in the recommended format
lstm_model.save(os.path.join(MODELS_PATH, "lstm_model.keras"), include_optimizer=True)
mlp_model.save(os.path.join(MODELS_PATH, "mlp_model.keras"), include_optimizer=True)

print("\n✅ Deep learning models and performance data saved successfully.")

235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

📌 MLP Model Model Evaluation:
MSE: 0.0126
RMSE: 0.1124
R-Squared: 0.9873
MAPE: 209.26%
MMRE: 2.0926
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

📌 LSTM Model Model Evaluation:
MSE: 0.0000
RMSE: 0.0029
R-Squared: 1.0000
MAPE: 7.60%
MMRE: 0.0760
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step

✅ Deep learning models and performance data saved successfully.
